In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('TRADES_CopyTr_90D_ROI.csv')

In [3]:
df.head()

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


In [5]:
df.isnull().sum()

Port_IDs         0
Trade_History    1
dtype: int64

In [6]:
import json

df['Trade_History'] = df['Trade_History'].fillna('')

# Function to safely parse JSON
def parse_json_safe(json_str):
    try:
        return json.loads(json_str)
    except (json.JSONDecodeError, TypeError) as e:
        return {}

df['Trade_History'] = df['Trade_History'].apply(lambda x: parse_json_safe(x))

In [7]:
# Function to calculate ROI (Return on Investment)
def calculate_roi(trade):
    if trade and 'realizedProfit' in trade:
        if trade['realizedProfit'] > 0:
            return trade['realizedProfit'] / trade['price']
        elif trade['realizedProfit'] < 0:
            return -1 * trade['price'] / trade['realizedProfit']
    return 0

df['ROI'] = df['Trade_History'].apply(lambda x: calculate_roi(x))

In [8]:
# Function to calculate Profit and Loss (PnL)
def calculate_pnl(trade):
    return trade['realizedProfit'] if trade and 'realizedProfit' in trade else 0

df['PnL'] = df['Trade_History'].apply(lambda x: calculate_pnl(x))

In [9]:
# Function to calculate Win Rate and Win Positions
def calculate_win_rate(trades):
    total_trades = len(trades)
    if total_trades == 0:
        return 0, 0
    profitable_trades = sum(trade['realizedProfit'] > 0 for trade in trades if 'realizedProfit' in trade)
    win_rate = profitable_trades / total_trades
    return win_rate, profitable_trades

win_rates, win_positions = [], []
for idx, row in df.iterrows():
    trades = row['Trade_History']
    win_rate, win_position = calculate_win_rate(trades)
    win_rates.append(win_rate)
    win_positions.append(win_position)

df['Win_Rate'] = win_rates
df['Win_Positions'] = win_positions

In [10]:
df['Total_Positions'] = df['Trade_History'].apply(len)

# Rank accounts based on calculated metrics
df_grouped = df.groupby('Port_IDs').agg({
    'ROI': 'sum',
    'PnL': 'sum',
    'Win_Rate': 'mean',
    'Win_Positions': 'sum',
    'Total_Positions': 'sum'
}).reset_index()

In [11]:
# Define weights for scoring
weights = {
    'ROI': 0.3,
    'PnL': 0.2,
    'Win_Rate': 0.15,
    'Win_Positions': 0.1,
    'Total_Positions': 0.05
}

# Calculate scores
df_grouped['Score'] = (df_grouped['ROI'] * weights['ROI'] +
                       df_grouped['PnL'] * weights['PnL'] +
                       df_grouped['Win_Rate'] * weights['Win_Rate'] +
                       df_grouped['Win_Positions'] * weights['Win_Positions'] +
                       df_grouped['Total_Positions'] * weights['Total_Positions'])

In [12]:
df_ranked = df_grouped.sort_values(by='Score', ascending=False).reset_index(drop=True)

top_20_accounts = df_ranked.head(20)

In [13]:
# Export calculated metrics to CSV
df_ranked.to_csv('financial_metrics.csv', index=False)

print("Top 20 Accounts:")
print(top_20_accounts['Port_IDs'])

Top 20 Accounts:
0     3672754654734989568
1     4017263283405469185
2     4019895412775450368
3     4020204877254599680
4     4021243448368889856
5     4021669203289716224
6     4022565861939831809
7     4022641794255717633
8     4023616625178799873
9     4023697433751327232
10    4023697881318718465
11    4026179081117855488
12    4027747397961573889
13    4028427053699127040
14    4028701921959171840
15    4029299190618134272
16    4029422834086627072
17    4017323550554338817
18    4017110277719148289
19    4029507714735307777
Name: Port_IDs, dtype: int64
